# Recipies based bot powered by llamaindex, Amazon Bedrock Titan Text Embeddings , AWS Bedrock Claude 3 Sonnet.

This notebook shows you how to use `ReAct` Agent over defined tools and RAG pipeline with Claude 3 Sonnet. Also added a reranker model (mixedbread-ai/mxbai-rerank-base-v1) that runs locally on the notebook instance.

In [7]:
!pip install llama-index
%pip install llama-index-llms-bedrock
%pip install llama-index-embeddings-bedrock
!pip uninstall pydantic -y
!pip install pydantic
%pip install sqlalchemy==2.0.21 --force-reinstall --quiet
%pip install llama-index-readers-elasticsearch
%pip install llama-index-vector-stores-opensearch

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Found existing installation: pydantic 2.7.1
Uninstalling pydantic-2.7.1:
  Successfully uninstalled pydantic-2.7.1
  Using cached pydantic-2.7.1-py3-none-any.whl.metadata (107 kB)
Using cached pydantic-2.7.1-py3-none-any.whl (409 kB)
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [8]:

from llama_index.embeddings.bedrock import BedrockEmbedding
from llama_index.core.postprocessor import LLMRerank



In [22]:
import json
from typing import Sequence, List
from llama_index.core.settings import Settings
from llama_index.llms.bedrock import Bedrock
from llama_index.embeddings.bedrock import BedrockEmbedding, Models

# ------------------------------------------------------------------------
# LlamaIndex - Amazon Bedrock


llm = Bedrock(model = "anthropic.claude-3-sonnet-20240229-v1:0")
embed_model = BedrockEmbedding(model = "amazon.titan-embed-text-v1")


Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 512


from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool

import nest_asyncio

nest_asyncio.apply()

In [23]:
import os
from os import getenv
from llama_index.core import SimpleDirectoryReader
from llama_index.vector_stores.opensearch import (
    OpensearchVectorStore,
    OpensearchVectorClient,
)
from llama_index.core import VectorStoreIndex, StorageContext

# http endpoint for your cluster (opensearch required for vector index usage)
endpoint = getenv("OPENSEARCH_ENDPOINT", "https://vpc-llamaindex-ank67ovnioagxtgqsqlq6iaqwa.us-west-2.es.amazonaws.com")
# index to demonstrate the VectorStore impl
idx = getenv("OPENSEARCH_INDEX", "gpt-index-recipies-titanv1")

In [24]:
%pip install torch sentence-transformers
from llama_index.core.postprocessor import SentenceTransformerRerank
postprocessor = SentenceTransformerRerank(
    model="mixedbread-ai/mxbai-rerank-base-v1", top_n=3
)

Note: you may need to restart the kernel to use updated packages.


In [25]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex


# OpensearchVectorClient stores text in this field by default
text_field = "content"
# OpensearchVectorClient stores embeddings in this field by default
embedding_field = "embedding"
# OpensearchVectorClient encapsulates logic for a
# single opensearch index with vector search enabled
client = OpensearchVectorClient(
    endpoint, idx, 1536, embedding_field=embedding_field, text_field=text_field
)
# initialize vector store
vector_store = OpensearchVectorStore(client)
storage_context = StorageContext.from_defaults(vector_store=vector_store)


# initialize vector store
vector_store = OpensearchVectorStore(client)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# load data from calories datasaet and create an index
calorie_docs = SimpleDirectoryReader(input_dir="./recipies-data/calories").load_data(num_workers=8)

# initialize vector store
vector_store = OpensearchVectorStore(client)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
# initialize an index using our sample data and the client we just created
calories_index = VectorStoreIndex.from_documents(
    documents=calorie_docs, storage_context=storage_context
)

calorie_engine = calories_index.as_query_engine(similarity_top_k=15, llm=llm, node_postprocessors=[postprocessor])


In [26]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex


# OpensearchVectorClient stores text in this field by default
text_field = "content"
# OpensearchVectorClient stores embeddings in this field by default
embedding_field = "embedding"
# OpensearchVectorClient encapsulates logic for a
# single opensearch index with vector search enabled
client = OpensearchVectorClient(
    endpoint, idx, 1536, embedding_field=embedding_field, text_field=text_field
)
# initialize vector store
vector_store = OpensearchVectorStore(client)
storage_context = StorageContext.from_defaults(vector_store=vector_store)


# initialize vector store
vector_store = OpensearchVectorStore(client)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# load data from calories datasaet and create an index
recipies_docs = SimpleDirectoryReader(input_dir="./recipies-data/recipies").load_data(num_workers=8)

# initialize vector store
vector_store = OpensearchVectorStore(client)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
# initialize an index using our sample data and the client we just created
recipies_index = VectorStoreIndex.from_documents(
    documents=recipies_docs, storage_context=storage_context
)

recipies_engine = recipies_index.as_query_engine(similarity_top_k=10, llm=llm,node_postprocessors=[postprocessor])


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7ff038e92b30>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7ff0e85a3c10>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7ff0e84ec130>


In [ ]:
# to be added once Bedrock Multi-Modal APIs are allowed. We will index images of dishes and do multi-modal RAG. 

#from llama_index.core.tools import QueryEngineTool, ToolMetadata
#from llama_index.core import SimpleDirectoryReader, VectorStoreIndex


# OpensearchVectorClient stores text in this field by default
#text_field = "content"
# OpensearchVectorClient stores embeddings in this field by default
#embedding_field = "embedding"
# OpensearchVectorClient encapsulates logic for a
# single opensearch index with vector search enabled
#client = OpensearchVectorClient(
#    endpoint, idx, 1024, embedding_field=embedding_field, text_field=text_field
#)
# initialize vector store
#vector_store = OpensearchVectorStore(client)
#storage_context = StorageContext.from_defaults(vector_store=vector_store)


# initialize vector store
#vector_store = OpensearchVectorStore(client)
#storage_context = StorageContext.from_defaults(vector_store=vector_store)

# load data from calories datasaet and create an index
#recipies_docs = SimpleDirectoryReader(input_dir="./recipies-data/recipies-pic").load_data(num_workers=8)

# initialize vector store
#vector_store = OpensearchVectorStore(client)
#storage_context = StorageContext.from_defaults(vector_store=vector_store)
# initialize an index using our sample data and the client we just created
#recipies_index = VectorStoreIndex.from_documents(
#    documents=recipies_docs, storage_context=storage_context
#)

#recipies_engine = recipies_index.as_query_engine(similarity_top_k=5, llm=llm)


In [27]:
# run query
#query_engine = index_cohere.as_query_engine(similarity_top_k=10, llm=llm, node_postprocessors=[postprocessor])
res = recipies_engine.query("How can we make pasta?")
res.response

'Here are the steps to make fresh pasta based on the context information provided:\n\n1. Make the pasta dough by whisking together salt and flour, making a well in the center, and cracking in eggs. Mix the eggs with a fork, then slowly incorporate the flour until a shaggy dough forms. Knead the dough on a lightly floured surface until smooth and elastic, about 5 minutes. Wrap and let rest for 1-2 hours.\n\n2. Divide the rested dough into 4 pieces. Working with one piece at a time, flatten it into a rectangle and pass it through a pasta maker, starting at the thickest setting. Fold and pass through again, then continue passing through without folding, adjusting to thinner settings each time until the sheet is 1/16 inch thick. \n\n3. Cut the thin pasta sheets crosswise into long noodles, about 8 inches long, for something like fettuccine or spaghetti. For ravioli, cut into 2-inch squares and fill with desired filling before sealing. For lasagna, leave the sheets long.\n\n4. Cook the fres

In [48]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=calorie_engine,
        metadata=ToolMetadata(
            name="calorie_metadata",
            description=(
                "Provides information about calories in different fruits and vegertables. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
        QueryEngineTool(
        query_engine=recipies_engine,
        metadata=ToolMetadata(
            name="recipies",
            description=(
                "Provides information about food recipies of various dishes. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

### With ReAct Agent

In [35]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

response = agent.chat("Give me some options to create a grill fish dish which has less than 500 calories? Think step by Step")
print(str(response))

Thought: The current language of the user is English. To provide options for a grilled fish dish with less than 500 calories, I will need information on calorie counts for different types of fish and recipes for grilled fish dishes.
Action: calorie_metadata
Action Input: {'input': 'What are some types of fish that are low in calories?'}
Observation: Based on the calorie information provided, some types of fish that are relatively low in calories per 100g serving include:

- Sole (86 cal)
- Squid (92 cal)  
- Triggerfish (93 cal)
- Redfish (94 cal)
- Monkfish (97 cal)
- Plaice (91 cal)
- Hake (71 cal)
- Zander (84 cal)
- Scampi (84 cal)
- Lobster (89 cal)
- Haddock (90 cal)

Fish like sole, squid, triggerfish, redfish, and monkfish all contain under 100 calories per 100g portion, making them good low-calorie seafood options.
Thought: The calorie metadata provided some good options for low-calorie fish. To create a full grilled fish dish under 500 calories, I will also need recipe inform

In [32]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

response = agent.chat("How many calories in sliced gouda cheese?")
print(str(response))

Thought: The current language of the user is: English. I need to use a tool to help me answer the question about the calories in sliced gouda cheese.
Action: calorie_metadata
Action Input: {'input': 'How many calories are in sliced gouda cheese?'}
Observation: According to the provided information, sliced Gouda cheese contains 356 calories per 100 grams.
Thought: The calorie metadata tool provided the information needed to answer the question about calories in sliced gouda cheese. I can now provide a final answer.
Answer: Sliced Gouda cheese contains 356 calories per 100 grams.
Sliced Gouda cheese contains 356 calories per 100 grams.


In [36]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

response = agent.chat("Give me some options to create a grill chicken dish which has less than 500 calories? Think step by Step")
print(str(response))

Thought: The current language of the user is English. To find options for a grilled chicken dish with less than 500 calories, I will first need to find the calorie information for grilled chicken.
Action: calorie_metadata
Action Input: {'input': 'How many calories are in 100g of grilled chicken breast?'}
Observation: Unfortunately, the provided context does not contain any information about the calorie content of grilled chicken breast. The context includes several recipes involving grilled chicken breasts, but does not provide nutritional details or calorie counts.
Thought: Okay, since I don't have calorie information for grilled chicken breast from the tools, I'll need to make an estimate based on typical nutritional values. A 100g serving of grilled chicken breast is usually around 165 calories.
Action: recipies
Action Input: {'input': 'What are some healthy side dishes or ingredients I can add to grilled chicken breast to make a dish under 500 calories?'}
Observation: Some healthy 

In [38]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

response = agent.chat("Give me some options to create a morning breakfast smoothie which has less than 500 calories? Think step by Step")
print(str(response))

Thought: The current language of the user is English. To create a low-calorie breakfast smoothie, I will need information on the calorie content of different fruits and vegetables that can be used in a smoothie.
Action: calorie_metadata
Action Input: {'input': 'What are some low-calorie fruits and vegetables that can be used in a breakfast smoothie?'}
Observation: Some low-calorie fruits and vegetables that can be used in a breakfast smoothie include spinach, cauliflower, carrots, cucumber, watermelon, mango, and persimmon. These ingredients are featured in several of the smoothie recipes provided, such as the "Watermelon Chia Smoothie" and the "Secret Mango Creamsicle Smoothie." Spinach, cauliflower, carrots, and cucumber are nutrient-dense vegetables that can add fiber, vitamins, and minerals to a smoothie without contributing excessive calories. Watermelon, mango, and persimmon are naturally sweet, low-calorie fruits that can provide flavor and nutrition to a smoothie.
Thought: The 

In [49]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

response = agent.chat("I want to make a cheesecake which has fruits as the topping and is not more than 2000 calories? Think Step by Step and recommend.")
print(str(response))

Thought: The current language of the user is English. To find a cheesecake recipe with fruits as topping and under 2000 calories, I should first look for cheesecake recipes with fruit toppings.
Action: recipies
Action Input: {'input': 'cheesecake recipes with fruit toppings'}
Observation: Here are a few cheesecake recipes with fruit toppings from the provided context:

1. Creole Cream Cheesecake with Caramel Apple Topping - This cheesecake has a graham cracker crust and is topped with a caramel apple mixture made with butter, sugars, honey, apples, lemon zest and juice, spices like cinnamon, cardamom and ginger.

2. Coconut Cheesecake with Passion Fruit Glaze - This coconut-flavored cheesecake has a passion fruit glaze made with passion fruit pulp, sugar and gelatin. It's also served with a tropical fruit compote containing mango, pineapple, star fruit and passion fruit.

3. Summer's Best Blueberry Sauce - While not a full cheesecake recipe, this blueberry sauce made with fresh blueber

In [54]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

response = agent.chat("Can you recommend me a cocktail drink which is based on Egg Nog? Also give a calorie count. think step bys step")
print(str(response))

Thought: The current language of the user is English. To recommend a cocktail drink based on eggnog and provide a calorie count, I will first need to find a suitable recipe using the recipies tool.
Action: recipies
Action Input: {'input': 'cocktail recipe with eggnog'}
Observation: Here is a cocktail recipe with eggnog from the provided context:

Uncle Angelo's Egg Nog
Ingredients:
- 6 eggs, separated
- 3/4 cup sugar
- 1 quart milk
- 1 pint cream
- 6 ounces bourbon
- 6 ounces spiced rum
- 1 whole nutmeg, for grating

Instructions:
1. Beat the egg yolks well until they turn light in color, adding 1/2 cup of the sugar as you beat. 
2. Add the milk, cream, and liquor.
3. Beat the egg whites with the remaining sugar until they peak. 
4. Fold the whipped egg whites into the yolk mixture.
5. Grate fresh nutmeg over the top before serving.

This rich and creamy eggnog cocktail is spiked with bourbon and spiced rum for an indulgent holiday treat.
Thought: The recipe provides helpful informatio